In [50]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999

In [22]:
sh = pd.read_csv("../../output/header_sims_info.tsv", sep="\t")
sinfo = pd.read_csv("../../output/sims_info.tsv", sep="\t", header=None, names=sh.columns)
sinfo['date'] = sinfo['date'].str.split('_', expand=True).iloc[:,[1,0]].agg('_'.join, axis=1)
sinfo = sinfo.round(15)
sinfo.rescf = sinfo.rescf.astype(str)
sinfo.sort_values(by=["date"], inplace=True)

In [23]:
n_trees = sinfo.edge.unique().size

In [24]:
sh.columns

Index(['edge', 'parent', 'N', 'gens', 'rescf', 'padding', 'delrate', 'delcoef',
       'posrate', 'poscoef', 'chr', 'start', 'end', 'win_id', 'win_len',
       'clen', 'padded_start', 'padded_end', 'L', 'mu', 'delprop', 'posprop',
       'siminterval', 'numid', 'rand_id', 'supnumid', 'sup_rand_id',
       'outfile_pre', 'exonfile', 'recfile', 'recfilehap', 'par_string',
       'date', 'log', 'poscoefdecayeps'],
      dtype='object')

In [28]:
np.unique(sinfo.sup_rand_id).shape

(114,)

In [56]:
diff_cols = ["delrate","delcoef","posrate","poscoef","mu", "poscoefdecayeps", "rescf", "win_len", "padding"]
# add poscoefdecayeps

In [57]:
ddf = sinfo[diff_cols+["sup_rand_id"]].copy()
ddf.drop_duplicates(subset=diff_cols, keep="last", inplace=True)

In [60]:
# asserting no parameter combo was duplicated
assert np.sum(ddf.duplicated(subset=diff_cols)) == 0

0

In [61]:
sup_has_all_branches = (ddf.groupby(by=['sup_rand_id']).size()<14)

In [62]:
g = sinfo.groupby(["rand_id"])
s = g.size()
not_all_edges = s[s<sinfo.edge.unique().size]

In [64]:
import os
root_dir = "../../output/"
not_all_tss = []
not_in_sinfo = []
for file in os.listdir(root_dir):
    fpath = root_dir+file
    if os.path.isdir(fpath):
        is_ts = [f.endswith("trees") for f in os.listdir(fpath)]
        if sum(is_ts) < n_trees:
            not_all_tss.append(file)
        if (sinfo.rand_id == file).sum() == 0:
            not_in_sinfo.append(file)

In [65]:
not_in_sinfo

['figs', 'backup', 'tmp', 'joined_stats']

In [ ]:
import shutil
for i in not_all_tss:
    if not any(sinfo[sinfo.rand_id == i].date>'30022021_0000') and (len(i) >10):
        #print(root_dir+i)
        shutil.rmtree(root_dir+i)
for i in not_in_sinfo:
    if len(i)>10:
        shutil.rmtree(root_dir+i)

In [ ]:
(sinfo.rand_id.isin(os.listdir(root_dir))).sum()

In [ ]:
sinfo[sinfo.rescf.astype(int)>1]